In [2]:
import numpy as np
import tensorflow as tf
mnist = tf.keras.datasets.mnist

INPUT_FEATURE = 'image'
NUM_CLASSES = 10

In [3]:
# Load training and eval data
(x_train, y_train),(x_test, y_test) = mnist.load_data()
train_data = np.asarray(x_train, dtype=np.float32)
eval_data = np.asarray(x_test, dtype=np.float32)
train_labels = np.asarray(y_train, dtype=np.int32)
eval_labels = np.asarray(y_test, dtype=np.int32)

In [4]:
train_data.shape

(60000, 28, 28)

In [5]:
# reshape images
# To have input as an image, we reshape images beforehand.
train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
eval_data = eval_data.reshape(eval_data.shape[0], 28, 28, 1)

In [6]:
def cnn_postmates_model_fn(features, labels, mode, params):
    """Model function for CNN."""
    # Input Layer
    input_layer = features

    # First convolutional Layer and pooling layer
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=None)
    batch_norm1 = tf.layers.batch_normalization(conv1)
    relu1 = tf.nn.relu(batch_norm1)
    #pool1 = tf.layers.max_pooling2d(inputs=relu1, pool_size=[2, 2], strides=2)

    # 1- Second convolutional Layer and pooling layer
    conv21 = tf.layers.conv2d(
        inputs=relu1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=None)
    batch_norm21= tf.layers.batch_normalization(conv21)
    relu21 = tf.nn.relu(batch_norm21)
    pool21 = tf.layers.max_pooling2d(inputs=relu21, pool_size=[2, 2], strides=2)

    # 2- Second convolutional Layer and pooling layer
    conv22 = tf.layers.conv2d(
        inputs=relu1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=None)
    batch_norm22= tf.layers.batch_normalization(conv22)
    relu22 = tf.nn.relu(batch_norm22)
    pool22 = tf.layers.max_pooling2d(inputs=relu22, pool_size=[2, 2], strides=2)

    # 1- third convolutional Layer and pooling layer
    conv31 = tf.layers.conv2d(
        inputs=pool21,
        filters=256,
        kernel_size=[5, 5],
        padding="same",
        activation=None)
    batch_norm31 = tf.layers.batch_normalization(conv31)
    relu31 = tf.nn.relu(batch_norm31)
    pool31 = tf.layers.max_pooling2d(inputs=relu31, pool_size=[2, 2], strides=2)

    # 2- third convolutional Layer and pooling layer
    conv32 = tf.layers.conv2d(
        inputs=pool22,
        filters=256,
        kernel_size=[5, 5],
        padding="same",
        activation=None)
    batch_norm32 = tf.layers.batch_normalization(conv32)
    relu32 = tf.nn.relu(batch_norm32)
    pool32 = tf.layers.max_pooling2d(inputs=relu32, pool_size=[2, 2], strides=2)

    comb3 = tf.concat([pool31, pool32], axis=2)

    # Flatten tensor into a batch of vectors
    comb3_flat = tf.layers.flatten(comb3)

    # 4 Dense Layer
    dense4 = tf.layers.dense(inputs=comb3_flat, units=1000, activation=tf.nn.relu)

    # 5 Dense Layer
    dense5 = tf.layers.dense(inputs=dense4, units=500, activation=tf.nn.relu)

    # Add dropout operation
    dropout = tf.layers.dropout(
        inputs=dense5, rate=0.4, training=(mode == tf.estimator.ModeKeys.TRAIN))

    # Logits layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    # PREDICT mode
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=predictions,
            export_outputs={
                'predict': tf.estimator.export.PredictOutput(predictions)
            })

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss =  params["loss_func"](labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer()
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [7]:
def serving_input_receiver_fn():
    """
    This is used to define inputs to serve the model.
    :return: ServingInputReciever
    """
    reciever_tensors = {
        # The size of input image is flexible.
        INPUT_FEATURE: tf.placeholder(tf.float32, [None, None, None, 1]),
    }

    # Convert give inputs to adjust to the model.
    features = {
        # Resize given images.
        INPUT_FEATURE: tf.image.resize_images(reciever_tensors[INPUT_FEATURE], [28, 28]),
    }
    return tf.estimator.export.ServingInputReceiver(receiver_tensors=reciever_tensors,
                                                    features=features)


In [8]:
# Create the Estimator
training_config = tf.estimator.RunConfig(
    model_dir="model_dir",
    save_summary_steps=20,
    save_checkpoints_steps=20)
classifier = tf.estimator.Estimator(
    model_fn=cnn_postmates_model_fn,
    model_dir="model_dir",
    config=training_config,
    params={"optimizer": tf.train.AdamOptimizer(),
            "loss_func":tf.losses.sparse_softmax_cross_entropy,
            "classes_count": 10})

INFO:tensorflow:Using config: {'_model_dir': 'model_dir', '_tf_random_seed': None, '_save_summary_steps': 20, '_save_checkpoints_steps': 20, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x102f1d7f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

In [10]:
# Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x=train_data,
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    classifier.train(
        input_fn=train_input_fn,
        steps=1,
        hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x=eval_data,
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = classifier.evaluate(input_fn=eval_input_fn,hooks=[logging_hook])
    print(eval_results)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into model_dir/model.ckpt.
INFO:tensorflow:probabilities = [[0.         0.         0.9910982  0.00000001 0.00000002 0.00000742
  0.00000187 0.         0.         0.00889244]
 [0.         0.         0.00000141 0.00000001 0.         0.00016107
  0.00000001 0.         0.         0.9998375 ]
 [0.         0.         0.3803987  0.00000001 0.         0.
  0.         0.         0.00009711 0.61950415]
 [0.         0.         0.9999943  0.         0.         0.
  0.         0.  

INFO:tensorflow:loss = 21.017338, step = 1
INFO:tensorflow:Saving checkpoints for 1 into model_dir/model.ckpt.
INFO:tensorflow:Loss for final step: 21.017338.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-12-15:15:16
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_dir/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:probabilities = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
INFO:tensorflow:probabilities = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (48.726 sec)
INFO:tensorflow:Finished evaluation at 2018-11-12-15:16:32
INFO:tensorflow:Saving dict for global step 1: accuracy = 0.0958, global_ste